In [1]:
import numpy as np
import pandas as pd
import jsonlines
from tqdm import tqdm
from joblib import Parallel, delayed
import category_encoders as ce

pd.set_option('display.max_columns', None)

/Users/mavillan/miniconda3/lib/python3.8/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def reduce_mem_usage(df, verbose=False):
    """
    Utility function to reduce the memory usage of pandas dataframes
    
    Parameters
    ----------
    df: pandas.Dataframe
    verbose: Boolean
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [15]:
def reduce_mem_usage(df, verbose=False):
    """
    Utility function to reduce the memory usage of pandas dataframes
    
    Parameters
    ----------
    df: pandas.Dataframe
    verbose: Boolean
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

***
## preparing train data

In [2]:
train = pd.read_parquet("../data/train_data.parquet")
train["date"] = pd.to_datetime(train["date"])
train["weekday"] = (train.date.dt.weekday+1).astype(str)
train

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,weekday
0,464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,1
1,464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,2
2,464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,3
3,464801,2021-02-04,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,4
4,464801,2021-02-05,1,156.78,REA,classic,fulfillment,free_shipping,1440.000000,5
...,...,...,...,...,...,...,...,...,...,...
37660274,129187,2021-03-31,0,22057.00,ARG,classic,drop_off,free_shipping,267.710767,3
37660275,6707,2021-03-31,0,26999.00,ARG,classic,cross_docking,free_shipping,266.083333,3
37660276,170355,2021-03-31,0,3400.00,ARG,classic,drop_off,paid_shipping,0.252633,3
37660277,246568,2021-03-31,0,6289.00,ARG,classic,fulfillment,free_shipping,135.416667,3


In [3]:
train.loc[:,["currency","listing_type","shipping_logistic_type","shipping_payment"]].nunique()

currency                  4
listing_type              2
shipping_logistic_type    3
shipping_payment          2
dtype: int64

In [4]:
encoder = ce.OneHotEncoder(cols=["weekday","listing_type","shipping_logistic_type","shipping_payment"], use_cat_names=True)
train = encoder.fit_transform(train)
train

/Users/mavillan/miniconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,sku,date,sold_quantity,current_price,currency,listing_type_classic,listing_type_premium,shipping_logistic_type_fulfillment,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_payment_free_shipping,shipping_payment_paid_shipping,minutes_active,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,464801,2021-02-01,0,156.78,REA,1,0,1,0,0,1,0,1440.000000,1,0,0,0,0,0,0
1,464801,2021-02-02,0,156.78,REA,1,0,1,0,0,1,0,1440.000000,0,1,0,0,0,0,0
2,464801,2021-02-03,0,156.78,REA,1,0,1,0,0,1,0,1440.000000,0,0,1,0,0,0,0
3,464801,2021-02-04,0,156.78,REA,1,0,1,0,0,1,0,1440.000000,0,0,0,1,0,0,0
4,464801,2021-02-05,1,156.78,REA,1,0,1,0,0,1,0,1440.000000,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37660274,129187,2021-03-31,0,22057.00,ARG,1,0,0,0,1,1,0,267.710767,0,0,1,0,0,0,0
37660275,6707,2021-03-31,0,26999.00,ARG,1,0,0,1,0,1,0,266.083333,0,0,1,0,0,0,0
37660276,170355,2021-03-31,0,3400.00,ARG,1,0,0,0,1,0,1,0.252633,0,0,1,0,0,0,0
37660277,246568,2021-03-31,0,6289.00,ARG,1,0,1,0,0,1,0,135.416667,0,0,1,0,0,0,0


In [5]:
test = pd.read_csv("../data/test_data.csv")
train = pd.merge(train, test, how="inner", on="sku")
train

,sku,date,sold_quantity,current_price,currency,listing_type_classic,listing_type_premium,shipping_logistic_type_fulfillment,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_payment_free_shipping,shipping_payment_paid_shipping,minutes_active,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,target_stock
0,464801,2021-02-01,0,156.78,REA,1,0,1,0,0,1,0,1440.000000,1,0,0,0,0,0,0,3
1,464801,2021-02-02,0,156.78,REA,1,0,1,0,0,1,0,1440.000000,0,1,0,0,0,0,0,3
2,464801,2021-02-03,0,156.78,REA,1,0,1,0,0,1,0,1440.000000,0,0,1,0,0,0,0,3
3,464801,2021-02-04,0,156.78,REA,1,0,1,0,0,1,0,1440.000000,0,0,0,1,0,0,0,3
4,464801,2021-02-05,1,156.78,REA,1,0,1,0,0,1,0,1440.000000,0,0,0,0,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31284422,129187,2021-03-31,0,22057.00,ARG,1,0,0,0,1,1,0,267.710767,0,0,1,0,0,0,0,8
31284423,6707,2021-03-31,0,26999.00,ARG,1,0,0,1,0,1,0,266.083333,0,0,1,0,0,0,0,30
31284424,170355,2021-03-31,0,3400.00,ARG,1,0,0,0,1,0,1,0.252633,0,0,1,0,0,0,0,3
31284425,246568,2021-03-31,0,6289.00,ARG,1,0,1,0,0,1,0,135.416667,0,0,1,0,0,0,0,2


In [6]:
cols_sum = [
    "listing_type_classic",
    "listing_type_premium",
    "shipping_logistic_type_fulfillment",
    "shipping_logistic_type_cross_docking",
    "shipping_logistic_type_drop_off",
    "shipping_payment_free_shipping",
    "shipping_payment_paid_shipping",
    "weekday_1",
    "weekday_2",
    "weekday_3",
    "weekday_4",
    "weekday_5",
    "weekday_6",
    "weekday_7",    
]
cols_mean = [
    "minutes_active",
]

def compute_features(df):
    target_stock = df.target_stock.values[0]
    start_index = df.index.min()
    end_index = df.index.max()

    if len(df) < 30:
        return None

    inventory_days = list()
    sum_values = list()
    mean_values = list()

    for i in df.index:
        if i+29 > end_index: break

        df_slice = df.loc[i:i+29]
        cumsum = df_slice.sold_quantity.values.cumsum()
        idxs = np.argwhere(cumsum >= target_stock)
        if len(idxs)==0:
            idays = np.inf
        else:
            idays = np.min(idxs) + 1

        inventory_days.append(idays)
        sum_values.append(df_slice[cols_sum].sum(axis=0).values)
        mean_values.append(df_slice[cols_mean].mean(axis=0).values)

    df = df.head(len(inventory_days)).copy()
    df["inventory_days"] = inventory_days
    df[cols_sum] = np.asarray(sum_values)
    df[cols_mean] = np.asarray(mean_values)
    
    return df

with Parallel(n_jobs=5) as parallel:
    delayed_func = delayed(compute_features)
    all_dfs = parallel(delayed_func(df) for _,df in tqdm(train.groupby("sku")))
    
# todo: filter nones & concat
# todo: seq_nbr

100%|██████████| 551472/551472 [1:53:25<00:00, 81.03it/s]  


In [14]:
train = pd.concat(filter(lambda x: x is not None, all_dfs), ignore_index=True)
train

,sku,date,sold_quantity,current_price,currency,listing_type_classic,listing_type_premium,shipping_logistic_type_fulfillment,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_payment_free_shipping,shipping_payment_paid_shipping,minutes_active,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,target_stock,inventory_days
0,0,2021-02-01,0,172.79,REA,30,0,0,30,0,30,0,1325.088333,5,5,4,4,4,4,4,1,inf
1,0,2021-02-02,0,172.79,REA,30,0,0,30,0,30,0,1373.088333,4,5,5,4,4,4,4,1,inf
2,0,2021-02-03,0,179.99,REA,30,0,0,30,0,30,0,1421.088333,4,4,5,5,4,4,4,1,inf
3,0,2021-02-04,0,179.99,REA,30,0,0,30,0,30,0,1440.000000,4,4,4,5,5,4,4,1,inf
4,0,2021-02-05,0,179.99,REA,30,0,0,30,0,30,0,1440.000000,4,4,4,4,5,5,4,1,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15524803,660914,2021-02-26,1,59.99,REA,0,30,30,0,0,0,30,1440.000000,4,4,4,4,5,5,4,5,8.0
15524804,660914,2021-02-27,0,59.99,REA,0,30,30,0,0,0,30,1440.000000,4,4,4,4,4,5,5,5,9.0
15524805,660914,2021-02-28,1,59.99,REA,0,30,30,0,0,0,30,1440.000000,5,4,4,4,4,4,5,5,8.0
15524806,660914,2021-03-01,0,59.99,REA,0,30,30,0,0,0,30,1440.000000,5,5,4,4,4,4,4,5,9.0


In [16]:
all_records = list()
with jsonlines.open('../data/items_static_metadata_full.jl') as reader:
    for obj in tqdm(reader):
        all_records.append(obj)
        
metadata = pd.DataFrame(all_records)
metadata.drop(["item_title"], axis=1, inplace=True)
metadata

660916it [00:06, 97446.26it/s] 


,item_domain_id,item_id,site_id,sku,product_id,product_family_id
0,MLB-SNEAKERS,492155,MLB,0,None,MLB15832732
1,MLB-SURFBOARD_RACKS,300279,MLB,1,None,None
2,MLM-NECKLACES,69847,MLM,2,None,None
3,MLM-RINGS,298603,MLM,3,None,None
4,MLB-WEBCAMS,345949,MLB,4,None,None
...,...,...,...,...,...,...
660911,MLB-CELLPHONE_PARTS,320792,MLB,660911,None,None
660912,MLM-AUTOMOTIVE_EMBLEMS,90441,MLM,660912,None,None
660913,MLB-SOFA_AND_FUTON_COVERS,202580,MLB,660913,None,None
660914,MLB-SNEAKERS,490874,MLB,660914,None,None


In [17]:
train = pd.merge(train, metadata, how="inner", on="sku")
train

,sku,date,sold_quantity,current_price,currency,listing_type_classic,listing_type_premium,shipping_logistic_type_fulfillment,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_payment_free_shipping,shipping_payment_paid_shipping,minutes_active,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,target_stock,inventory_days,item_domain_id,item_id,site_id,product_id,product_family_id
0,0,2021-02-01,0,172.79,REA,30,0,0,30,0,30,0,1325.088333,5,5,4,4,4,4,4,1,inf,MLB-SNEAKERS,492155,MLB,None,MLB15832732
1,0,2021-02-02,0,172.79,REA,30,0,0,30,0,30,0,1373.088333,4,5,5,4,4,4,4,1,inf,MLB-SNEAKERS,492155,MLB,None,MLB15832732
2,0,2021-02-03,0,179.99,REA,30,0,0,30,0,30,0,1421.088333,4,4,5,5,4,4,4,1,inf,MLB-SNEAKERS,492155,MLB,None,MLB15832732
3,0,2021-02-04,0,179.99,REA,30,0,0,30,0,30,0,1440.000000,4,4,4,5,5,4,4,1,inf,MLB-SNEAKERS,492155,MLB,None,MLB15832732
4,0,2021-02-05,0,179.99,REA,30,0,0,30,0,30,0,1440.000000,4,4,4,4,5,5,4,1,inf,MLB-SNEAKERS,492155,MLB,None,MLB15832732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15524803,660914,2021-02-26,1,59.99,REA,0,30,30,0,0,0,30,1440.000000,4,4,4,4,5,5,4,5,8.0,MLB-SNEAKERS,490874,MLB,None,None
15524804,660914,2021-02-27,0,59.99,REA,0,30,30,0,0,0,30,1440.000000,4,4,4,4,4,5,5,5,9.0,MLB-SNEAKERS,490874,MLB,None,None
15524805,660914,2021-02-28,1,59.99,REA,0,30,30,0,0,0,30,1440.000000,5,4,4,4,4,4,5,5,8.0,MLB-SNEAKERS,490874,MLB,None,None
15524806,660914,2021-03-01,0,59.99,REA,0,30,30,0,0,0,30,1440.000000,5,5,4,4,4,4,4,5,9.0,MLB-SNEAKERS,490874,MLB,None,None


In [18]:
train = reduce_mem_usage(train, verbose=True)
train.to_parquet("../data/train-m2.parquet", index=False)

Mem. usage decreased to 1450.95 Mb (56.2% reduction)


***
## preparing test data

In [7]:
cols = ["sku","current_price", "currency", "listing_type", "shipping_logistic_type", 
        "shipping_payment", "minutes_active", "available", "item_domain_id", 
        "item_id", "site_id", "product_id", "product_family_id"]

def get_train_feats(df, look_back=7):
    row = dict()
    for col in cols:
        counts = df.tail(look_back).loc[:,col].value_counts()
        if len(counts) == 0:
            row[col] = None 
        else:
            row[col] = counts.index[0]  
    return row

In [8]:
all_rows = list()
    
with Parallel(n_jobs=6) as parallel:
    delayed_func = delayed(get_train_feats)
    all_rows = parallel(delayed_func(df) for _,df in tqdm(train.groupby("sku")))

train_feats = pd.DataFrame(all_rows)
train_feats

100%|██████████| 660916/660916 [27:12<00:00, 404.86it/s] 


,sku,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,available,item_domain_id,item_id,site_id,product_id,product_family_id
0,0,179.99,REA,classic,fulfillment,free_shipping,1440.0,1,MLB-SNEAKERS,492155,MLB,None,MLB15832732
1,1,135.90,REA,premium,fulfillment,free_shipping,0.0,1,MLB-SURFBOARD_RACKS,300279,MLB,None,None
2,2,219.00,MEX,premium,fulfillment,paid_shipping,1440.0,1,MLM-NECKLACES,69847,MLM,None,None
3,3,399.00,MEX,premium,fulfillment,free_shipping,1440.0,1,MLM-RINGS,298603,MLM,None,None
4,4,118.00,REA,premium,fulfillment,free_shipping,0.0,1,MLB-WEBCAMS,345949,MLB,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
660911,660911,12.90,REA,classic,fulfillment,paid_shipping,1440.0,1,MLB-CELLPHONE_PARTS,320792,MLB,None,None
660912,660912,99.00,MEX,classic,fulfillment,paid_shipping,1440.0,1,MLM-AUTOMOTIVE_EMBLEMS,90441,MLM,None,None
660913,660913,132.39,REA,premium,fulfillment,free_shipping,1440.0,1,MLB-SOFA_AND_FUTON_COVERS,202580,MLB,None,None
660914,660914,59.99,REA,premium,fulfillment,paid_shipping,1440.0,1,MLB-SNEAKERS,490874,MLB,None,None


In [9]:
test = pd.read_csv("../data/test_data.csv")
test = pd.merge(test, train_feats, how="left", on="sku")
test

,sku,target_stock,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,available,item_domain_id,item_id,site_id,product_id,product_family_id
0,464801,3,169.99,REA,classic,fulfillment,free_shipping,1440.000000,1,MLB-NEBULIZERS,344151,MLB,MLB9838512,MLB9838510
1,645793,4,164.99,REA,classic,fulfillment,free_shipping,1440.000000,1,MLB-NEBULIZERS,438135,MLB,None,MLB9838510
2,99516,8,22.90,REA,premium,fulfillment,paid_shipping,1440.000000,1,MLB-ADHESIVE_TAPES,221252,MLB,None,None
3,538100,8,28.10,REA,premium,fulfillment,paid_shipping,1440.000000,1,MLB-SCHOOL_AND_OFFICE_GLUES,62099,MLB,None,None
4,557191,10,49.90,REA,premium,fulfillment,paid_shipping,1440.000000,1,MLB-DECORATIVE_VINYLS,168198,MLB,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551467,129187,8,22057.00,ARG,classic,drop_off,free_shipping,267.710767,1,MLA-RADIO_BASE_STATIONS,408559,MLA,None,None
551468,6707,30,26999.00,ARG,classic,cross_docking,free_shipping,266.083333,1,MLA-PRINTERS,193784,MLA,MLA15159034,MLA15159032
551469,170355,3,3400.00,ARG,classic,drop_off,paid_shipping,0.252633,1,MLA-WRISTWATCHES,110276,MLA,None,None
551470,246568,2,6289.00,ARG,classic,fulfillment,free_shipping,135.416667,1,MLA-HARD_DRIVES_AND_SSDS,456892,MLA,MLA15697725,MLA15697724


In [10]:
all_dfs = list()

for date in pd.date_range("2021-04-01", "2021-04-30"):
    _test = test.copy(deep=True)
    _test["date"] = date
    all_dfs.append(_test)
    
test = pd.concat(all_dfs, ignore_index=True)
test

,sku,target_stock,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,available,item_domain_id,item_id,site_id,product_id,product_family_id,date
0,464801,3,169.99,REA,classic,fulfillment,free_shipping,1440.000000,1,MLB-NEBULIZERS,344151,MLB,MLB9838512,MLB9838510,2021-04-01
1,645793,4,164.99,REA,classic,fulfillment,free_shipping,1440.000000,1,MLB-NEBULIZERS,438135,MLB,None,MLB9838510,2021-04-01
2,99516,8,22.90,REA,premium,fulfillment,paid_shipping,1440.000000,1,MLB-ADHESIVE_TAPES,221252,MLB,None,None,2021-04-01
3,538100,8,28.10,REA,premium,fulfillment,paid_shipping,1440.000000,1,MLB-SCHOOL_AND_OFFICE_GLUES,62099,MLB,None,None,2021-04-01
4,557191,10,49.90,REA,premium,fulfillment,paid_shipping,1440.000000,1,MLB-DECORATIVE_VINYLS,168198,MLB,None,None,2021-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16544155,129187,8,22057.00,ARG,classic,drop_off,free_shipping,267.710767,1,MLA-RADIO_BASE_STATIONS,408559,MLA,None,None,2021-04-30
16544156,6707,30,26999.00,ARG,classic,cross_docking,free_shipping,266.083333,1,MLA-PRINTERS,193784,MLA,MLA15159034,MLA15159032,2021-04-30
16544157,170355,3,3400.00,ARG,classic,drop_off,paid_shipping,0.252633,1,MLA-WRISTWATCHES,110276,MLA,None,None,2021-04-30
16544158,246568,2,6289.00,ARG,classic,fulfillment,free_shipping,135.416667,1,MLA-HARD_DRIVES_AND_SSDS,456892,MLA,MLA15697725,MLA15697724,2021-04-30


In [13]:
test = reduce_mem_usage(test, verbose=True)
test.to_parquet("../data/test-m2.parquet", index=False)

Mem. usage decreased to 1435.77 Mb (24.2% reduction)


***